## Read Data

In [52]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="serhii",
    password="3244566Rko"
)

cur = conn.cursor()

with open("../work/queries/count_queries_with_cast_members.sql", 'r') as file:
    sql_script = file.read()

cur.execute(sql_script)

records = cur.fetchall()

for record in records:
    print(record)

cur.close()
conn.close()

('Akiva Goldsman', 1, datetime.date(2006, 5, 24))
('Alfred Molina', 2, datetime.date(2006, 3, 21))
('Alfred Molina', 2, datetime.date(2006, 5, 5))
('Alfred Molina', 1, datetime.date(2006, 5, 22))
('Alfred Molina', 1, datetime.date(2006, 5, 26))
('Andy Robb', 5, datetime.date(2006, 3, 23))
('Andy Robb', 4, datetime.date(2006, 3, 29))
('Andy Robb', 3, datetime.date(2006, 4, 12))
('Andy Robb', 1, datetime.date(2006, 5, 9))
('Andy Robb', 1, datetime.date(2006, 5, 11))
('Andy Skinner', 1, datetime.date(2006, 4, 25))
('Andy Skinner', 1, datetime.date(2006, 5, 26))
('Audrey Tautou', 14, datetime.date(2006, 3, 6))
('Audrey Tautou', 5, datetime.date(2006, 3, 12))
('Audrey Tautou', 1, datetime.date(2006, 4, 24))
('Audrey Tautou', 1, datetime.date(2006, 4, 25))
('Audrey Tautou', 13, datetime.date(2006, 4, 30))
('Audrey Tautou', 1, datetime.date(2006, 5, 3))
('Audrey Tautou', 1, datetime.date(2006, 5, 7))
('Audrey Tautou', 2, datetime.date(2006, 5, 9))
('Audrey Tautou', 2, datetime.date(2006, 5, 1

## Visualisation of Data

In [54]:
import pandas as pd
import plotly.graph_objects as go
import datetime

# Предположим, что данные уже есть в переменной records
# Пример: records = [('Akiva Goldsman', 1, datetime.date(2006, 5, 24)), ...]

# Создание DataFrame
df = pd.DataFrame(records, columns=['actor', 'queries', 'date'])

# Группировка данных по датам и актерам
grouped_df = df.groupby(['date', 'actor']).sum().reset_index()

# Получение уникальных актеров
actors = grouped_df['actor'].unique()

# Подготовка данных для интерактивного графика
data = []
dates = grouped_df['date'].unique()
bottom = pd.Series([0] * len(dates), index=dates)

# Словарь для хранения данных по датам
date_actor_dict = {date: {} for date in dates}

# Заполнение словаря
for actor in actors:
    actor_data = grouped_df[grouped_df['actor'] == actor]
    actor_data = actor_data.set_index('date')['queries']
    actor_data = actor_data.reindex(dates, fill_value=0)
    
    for date, queries in actor_data.items():
        if queries > 0:
            date_actor_dict[date][actor] = queries

# Подготовка данных для отображения
for actor in actors:
    actor_data = grouped_df[grouped_df['actor'] == actor]
    actor_data = actor_data.set_index('date')['queries']
    actor_data = actor_data.reindex(dates, fill_value=0)
    
    # Подготовка текста для hover
    hover_text = []
    for date in actor_data.index:
        if actor_data[date] > 0:
            total_queries = sum(date_actor_dict[date].values())
            percent = (actor_data[date] / total_queries) * 100 if total_queries != 0 else 0
            hover_text.append(f"{actor}: {actor_data[date]} ({percent:.2f}%)")
        else:
            hover_text.append("")

    data.append(go.Bar(
        x=actor_data.index,
        y=actor_data,
        name=actor,
        text=hover_text,
        hoverinfo='text',
        marker=dict(line=dict(width=0.5))
    ))
    
    
print(data['text'])


TypeError: list indices must be integers or slices, not str

In [51]:

# Добавление вертикальной линии для даты выхода фильма "Код ДаВинчи"
release_date = datetime.date(2006, 5, 19)
release_line = go.Scatter(
    x=[release_date, release_date],
    y=[0, max(grouped_df['queries'])],
    mode='lines',
    name='The Da Vinci Code Release',
    line=dict(color='red', width=2, dash='dash'),
    hoverinfo='x'
)

# Настройки макета графика
layout = go.Layout(
    title='Total Queries per Day by Actor',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Total Queries'),
    barmode='stack',
    hovermode='x unified',
    height=700  # Увеличение высоты графика в 1.5 раза
)

# Создание фигуры и добавление данных
fig = go.Figure(data=data, layout=layout)
fig.add_trace(release_line)

# Отображение графика
fig.show()
